# HealthBench-BR - Teste com 20 Primeiras Perguntas

Este notebook testa os modelos ativos configurados em `providers.json` usando as primeiras 20 perguntas do dataset.

In [9]:
import json
import os
import sys
import asyncio
from typing import List, Dict
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

# Carrega variáveis do .env sobrescrevendo as já existentes
load_dotenv(override=True)

# 🔍 Debug: imprime variáveis sensíveis carregadas (só nomes, não valores)
print("\n🔍 Variáveis carregadas do .env:")
for key in ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_DEFAULT_REGION", 
            "AWS_BEARER_TOKEN_BEDROCK", "OPENAI_API_KEY", "MARITACA_API_KEY"]:
    value = os.getenv(key)
    if value:
        masked = value[:4] + "..." if len(value) > 8 else "***"
        print(f"   {key} = {masked}")
    else:
        print(f"   {key} = ❌ Não definida")

# Add src to path
sys.path.insert(0, os.path.dirname(os.path.abspath('.')))

from src.providers.base import ProviderConfig
from src.providers.factory import ProviderFactory
from src.dataset.loader import DatasetLoader
from src.evaluation.evaluator import Evaluator
from src.reports.generator import ReportGenerator, EvaluationResult


🔍 Variáveis carregadas do .env:
   AWS_ACCESS_KEY_ID = ❌ Não definida
   AWS_SECRET_ACCESS_KEY = ❌ Não definida
   AWS_DEFAULT_REGION = ❌ Não definida
   AWS_BEARER_TOKEN_BEDROCK = ABSK...
   OPENAI_API_KEY = ❌ Não definida
   MARITACA_API_KEY = 1000...


## Configuração dos Providers

Carregamos os providers ativos do arquivo `providers.json`.

In [12]:
def load_active_providers(providers_file: str = "providers.json") -> List[Dict]:
    """Load active providers from JSON configuration file with environment and connectivity checks"""
    with open(providers_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # Filter only active providers
    candidate_providers = [p for p in config['providers'] if p.get('active', False)]
    
    print(f"Providers candidatos encontrados: {len(candidate_providers)}")
    
    validated_providers = []
    
    for provider in candidate_providers:
        print(f"\n🔍 Validando {provider['name']} ({provider['type']}: {provider['model']})...")
        
        # Check environment variables and connectivity
        is_valid = validate_provider_config(provider)
        
        if is_valid:
            validated_providers.append(provider)
            print(f"   ✅ {provider['name']} - Configuração válida")
        else:
            print(f"   ❌ {provider['name']} - Configuração inválida (será ignorado)")
    
    print(f"\n📊 Resumo da validação:")
    print(f"   Providers candidatos: {len(candidate_providers)}")
    print(f"   Providers válidos: {len(validated_providers)}")
    
    if validated_providers:
        print(f"\n✅ Providers prontos para teste:")
        for provider in validated_providers:
            print(f"   - {provider['name']} ({provider['type']}: {provider['model']})")
    
    return validated_providers, config['default_settings']


def validate_provider_config(provider: Dict) -> bool:
    """Validate provider configuration including environment variables and connectivity"""
    provider_type = provider['type']
    
    try:
        # Check environment variables based on provider type
        if provider_type == 'openai':
            return validate_openai_config(provider)
        elif provider_type == 'maritaca':
            return validate_maritaca_config(provider)
        elif provider_type == 'ollama':
            return validate_ollama_config(provider)
        elif provider_type == 'aws_bedrock':
            return validate_bedrock_config(provider)
        else:
            print(f"      ⚠️ Tipo de provider desconhecido: {provider_type}")
            return False
            
    except Exception as e:
        print(f"      ❌ Erro na validação: {e}")
        return False


def validate_openai_config(provider: Dict) -> bool:
    """Validate OpenAI provider configuration"""
    api_key = provider.get('api_key', '')
    
    # Resolve environment variable
    if api_key.startswith('${') and api_key.endswith('}'):
        env_var = api_key[2:-1]
        actual_key = os.getenv(env_var)
        if not actual_key:
            print(f"      ❌ Variável de ambiente {env_var} não encontrada")
            return False
        print(f"      ✅ Variável de ambiente {env_var} encontrada")
    elif not api_key:
        print(f"      ❌ API key não configurada")
        return False
    
    # Test connectivity (simple check)
    base_url = provider.get('base_url', 'https://api.openai.com')
    try:
        import requests
        response = requests.get(f"{base_url}/v1/models", timeout=10, headers={
            'Authorization': f"Bearer {actual_key if 'actual_key' in locals() else 'test'}"
        })
        if response.status_code in [200, 401]:  # 401 means auth failed but service is reachable
            print(f"      ✅ Conectividade com OpenAI OK")
            return True
        else:
            print(f"      ⚠️ Resposta inesperada da API: {response.status_code}")
            return False
    except Exception as e:
        print(f"      ⚠️ Não foi possível testar conectividade: {e}")
        return True  # Allow to proceed anyway


def validate_maritaca_config(provider: Dict) -> bool:
    """Validate Maritaca provider configuration"""
    api_key = provider.get('api_key', '')
    
    # Resolve environment variable
    if api_key.startswith('${') and api_key.endswith('}'):
        env_var = api_key[2:-1]
        actual_key = os.getenv(env_var)
        if not actual_key:
            print(f"      ❌ Variável de ambiente {env_var} não encontrada")
            return False
        print(f"      ✅ Variável de ambiente {env_var} encontrada")
    elif not api_key:
        print(f"      ❌ API key não configurada")
        return False
    
    # Test connectivity
    base_url = provider.get('base_url', 'https://chat.maritaca.ai')
    try:
        import requests
        response = requests.get(base_url, timeout=10)
        if response.status_code == 200:
            print(f"      ✅ Conectividade com Maritaca OK")
            return True
        else:
            print(f"      ⚠️ Resposta inesperada: {response.status_code}")
            return False
    except Exception as e:
        print(f"      ⚠️ Não foi possível testar conectividade: {e}")
        return True  # Allow to proceed anyway


def validate_ollama_config(provider: Dict) -> bool:
    """Validate Ollama provider configuration"""
    base_url = provider.get('base_url', 'http://localhost:11434')
    model = provider.get('model')
    
    # Test if Ollama is running
    try:
        import requests
        response = requests.get(f"{base_url}/api/tags", timeout=10)
        if response.status_code == 200:
            print(f"      ✅ Ollama está executando em {base_url}")
            
            # Check if model is available
            models = response.json()
            available_models = [m['name'] for m in models.get('models', [])]
            
            if model in available_models:
                print(f"      ✅ Modelo {model} está disponível")
                return True
            else:
                print(f"      ❌ Modelo {model} não encontrado")
                print(f"      💡 Modelos disponíveis: {available_models[:3]}{'...' if len(available_models) > 3 else ''}")
                return False
        else:
            print(f"      ❌ Ollama não está respondendo (código {response.status_code})")
            return False
    except Exception as e:
        print(f"      ❌ Erro ao conectar com Ollama: {e}")
        print(f"      💡 Verifique se Ollama está executando em {base_url}")
        return False


def validate_bedrock_config(provider: Dict) -> bool:
    """Validate AWS Bedrock provider configuration"""
    bearer_token = provider.get('aws_bearer_token', '')
    region = provider.get('region', 'us-east-1')
    
    # Check bearer token environment variable
    if bearer_token.startswith('${') and bearer_token.endswith('}'):
        env_var = bearer_token[2:-1]
        actual_token = os.getenv(env_var)
        if not actual_token:
            print(f"      ❌ Variável de ambiente {env_var} não encontrada")
            return False
        print(f"      ✅ Variável de ambiente {env_var} encontrada")

        # Se o bearer token existe, já é suficiente
        os.environ['AWS_BEARER_TOKEN_BEDROCK'] = actual_token
        print(f"      ✅ Bearer token configurado para Bedrock")
        return True   # ← retorno antecipado se token está ok
    
    # Caso não exista bearer token, verifica credenciais tradicionais
    aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
    aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
    
    if not aws_access_key and not aws_secret_key:
        print(f"      ⚠️ Credenciais AWS não encontradas no ambiente")
        print(f"      💡 Tentando usar perfil padrão AWS...")
        
        try:
            import boto3
            session = boto3.Session()
            credentials = session.get_credentials()
            if credentials:
                print(f"      ✅ Credenciais AWS encontradas no perfil")
                return True
            else:
                print(f"      ❌ Nenhuma credencial AWS disponível")
                return False
        except Exception as e:
            print(f"      ❌ Erro ao verificar credenciais AWS: {e}")
            return False
    else:
        print(f"      ✅ Credenciais AWS encontradas no ambiente")
        return True


# Load providers with validation
active_providers, default_settings = load_active_providers()

if not active_providers:
    raise ValueError("\n❌ Nenhum provider válido encontrado!\n\n" +
                    "💡 Possíveis soluções:\n" +
                    "   1. Verifique as variáveis de ambiente necessárias\n" +
                    "   2. Certifique-se de que o Ollama está executando (se aplicável)\n" +
                    "   3. Configure as credenciais AWS (se aplicável)\n" +
                    "   4. Verifique as configurações em providers.json")

Providers candidatos encontrados: 3

🔍 Validando MedGemma-27B-Q8 (ollama: jwang580/medgemma_27b_q8_0:latest)...
      ✅ Ollama está executando em http://localhost:11434
      ✅ Modelo jwang580/medgemma_27b_q8_0:latest está disponível
   ✅ MedGemma-27B-Q8 - Configuração válida

🔍 Validando Claude-Sonnet-4-Bedrock (aws_bedrock: global.anthropic.claude-sonnet-4-20250514-v1:0)...
      ✅ Variável de ambiente AWS_BEARER_TOKEN_BEDROCK encontrada
      ✅ Bearer token configurado para Bedrock
   ✅ Claude-Sonnet-4-Bedrock - Configuração válida

🔍 Validando Maritaca-Sabiazinho-3 (maritaca: sabiazinho-3)...
      ✅ Variável de ambiente MARITACA_API_KEY encontrada
      ✅ Conectividade com Maritaca OK
   ✅ Maritaca-Sabiazinho-3 - Configuração válida

📊 Resumo da validação:
   Providers candidatos: 3
   Providers válidos: 3

✅ Providers prontos para teste:
   - MedGemma-27B-Q8 (ollama: jwang580/medgemma_27b_q8_0:latest)
   - Claude-Sonnet-4-Bedrock (aws_bedrock: global.anthropic.claude-sonnet-4-202

## Carregamento do Dataset

Carregamos apenas as primeiras 20 perguntas do dataset para teste.

In [13]:
# Load dataset (first 20 questions)
print("Carregando dataset...")
dataset = DatasetLoader.load_dataset("benchmark_perguntas_unificado.json")
test_dataset = dataset[:20]  # Only first 20 questions

print(f"Dataset carregado: {len(test_dataset)} perguntas (primeiras 20)")
print(f"Arquivos únicos: {len(set(q.arquivo for q in test_dataset))}")
print(f"Títulos únicos: {len(set(q.titulo for q in test_dataset))}")

# Show sample questions
print("\nExemplo de perguntas:")
for i, question in enumerate(test_dataset[:3]):
    print(f"\n{i+1}. [{question.arquivo}] {question.titulo}")
    print(f"   Pergunta: {question.pergunta[:100]}...")
    print(f"   Resposta esperada: {question.esperado}")

Carregando dataset...
Dataset carregado: 20 perguntas (primeiras 20)
Arquivos únicos: 2
Títulos únicos: 2

Exemplo de perguntas:

1. [V_PCDT_-_Vasculite__-_Atualizado.pdf] Protocolo clínico e diretrizes terapêuticas da vasculite associada aos anticorpos anti-citoplasma de neutrófilos
   Pergunta: No que diz respeito ao protocolo clínico "Protocolo Clínico e Diretrizes Terapêuticas da Vasculite A...
   Resposta esperada: Verdadeiro

2. [V_PCDT_-_Vasculite__-_Atualizado.pdf] Protocolo clínico e diretrizes terapêuticas da vasculite associada aos anticorpos anti-citoplasma de neutrófilos
   Pergunta: No que diz respeito ao protocolo clínico "Protocolo Clínico e Diretrizes Terapêuticas da Vasculite A...
   Resposta esperada: Falso

3. [V_PCDT_-_Vasculite__-_Atualizado.pdf] Protocolo clínico e diretrizes terapêuticas da vasculite associada aos anticorpos anti-citoplasma de neutrófilos
   Pergunta: No que diz respeito ao protocolo clínico "Protocolo Clínico e Diretrizes Terapêuticas da Vascul

## Função para Criar Providers

Função auxiliar para instanciar providers baseado na configuração.

In [16]:
def create_provider_from_config(provider_config: Dict):
    """Create a provider instance from configuration"""
    from src.providers.maritaca import MaritacaProvider
    from src.providers.openai_provider import OpenAIProvider
    from src.providers.ollama import OllamaProvider
    from src.providers.bedrock import BedrockProvider
    
    # Create base config
    config = ProviderConfig(
        model_name=provider_config['model'],
        temperature=provider_config.get('temperature', 0.0),
        max_tokens=provider_config.get('max_tokens', 12000),
        api_key=provider_config.get('api_key'),
        base_url=provider_config.get('base_url'),
        timeout=provider_config.get('timeout', 120)
    )
    
    # Create provider based on type
    provider_type = provider_config['type']
    
    if provider_type == 'maritaca':
        return MaritacaProvider(config)
    elif provider_type == 'openai':
        return OpenAIProvider(config)
    elif provider_type == 'ollama':
        return OllamaProvider(config)
    elif provider_type == 'aws_bedrock':
        return BedrockProvider(
            config,
            region_name=provider_config.get('region', 'us-east-1'),
            aws_bearer_token=provider_config.get('aws_bearer_token')
        )
    else:
        raise ValueError(f"Tipo de provider não suportado: {provider_type}")

print("Função create_provider_from_config definida.")

Função create_provider_from_config definida.


## Execução dos Testes

Executamos a avaliação para cada provider ativo.

In [17]:
async def run_evaluation_for_provider(provider_config: Dict, dataset: List, parallelism: int = 3):
    """Run evaluation for a single provider"""
    print(f"\n{'='*60}")
    print(f"Testando: {provider_config['name']} ({provider_config['model']})")
    print(f"{'='*60}")
    
    try:
        # Create provider
        provider = create_provider_from_config(provider_config)
        
        # Create evaluator with reduced parallelism for testing
        evaluator = Evaluator(provider, parallelism=parallelism)
        
        # Run evaluation
        start_time = datetime.now()
        results = await evaluator.evaluate(dataset, show_progress=True)
        end_time = datetime.now()
        
        # Calculate metrics
        total = len(results)
        correct = sum(1 for r in results if r.correta)
        accuracy = correct / total if total > 0 else 0
        duration = (end_time - start_time).total_seconds()
        
        print(f"\n📊 Resultados para {provider_config['name']}:")
        print(f"   Total de perguntas: {total}")
        print(f"   Acertos: {correct}")
        print(f"   Acurácia: {accuracy:.2%}")
        print(f"   Tempo: {duration:.1f}s")
        
        return {
            'provider_name': provider_config['name'],
            'provider_type': provider_config['type'],
            'model': provider_config['model'],
            'total': total,
            'correct': correct,
            'accuracy': accuracy,
            'duration_seconds': duration,
            'results': results
        }
        
    except Exception as e:
        print(f"❌ Erro ao testar {provider_config['name']}: {e}")
        import traceback
        traceback.print_exc()
        return None

print("Função run_evaluation_for_provider definida.")

Função run_evaluation_for_provider definida.


## Execução dos Testes para Todos os Providers Ativos

In [18]:
# Run tests for all active providers
all_results = []

for provider_config in active_providers:
    result = await run_evaluation_for_provider(provider_config, test_dataset, parallelism=3)
    if result:
        all_results.append(result)

print(f"\n🎉 Testes concluídos! {len(all_results)} providers testados com sucesso.")


Testando: MedGemma-27B-Q8 (jwang580/medgemma_27b_q8_0:latest)
Total de perguntas a avaliar: 20


Batch 1:   0%|                                                                                        | 0/3 [00:00<?, ?it/s]/Users/filipelopes/Desktop/Development/notebooks/noharm-maritaca/healthbench-br/src/providers/ollama.py:16: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  return Ollama(
Batch 1: 100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [01:09<00:00, 23.17s/it]


Acurácia parcial: 2/3 = 0.667


Batch 2: 100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:41<00:00, 13.93s/it]


Acurácia parcial: 3/6 = 0.500


...

Acurácia parcial: 0/15 = 0.000


Batch 6: 100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 21.13it/s]


Acurácia parcial: 0/18 = 0.000


Batch 7: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.21it/s]

Acurácia parcial: 0/20 = 0.000

📊 Resultados para Maritaca-Sabiazinho-3:
   Total de perguntas: 20
   Acertos: 0
   Acurácia: 0.00%
   Tempo: 1.5s

🎉 Testes concluídos! 3 providers testados com sucesso.


## Comparação de Resultados

Visualizamos uma tabela comparativa dos resultados.

In [19]:
if all_results:
    # Create comparison DataFrame
    comparison_data = []
    for result in all_results:
        comparison_data.append({
            'Provider': result['provider_name'],
            'Tipo': result['provider_type'],
            'Modelo': result['model'],
            'Total': result['total'],
            'Acertos': result['correct'],
            'Acurácia': f"{result['accuracy']:.2%}",
            'Tempo (s)': f"{result['duration_seconds']:.1f}"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print("📋 COMPARAÇÃO DE RESULTADOS")
    print("=" * 80)
    print(comparison_df.to_string(index=False))
    
    # Find best performer
    best_result = max(all_results, key=lambda x: x['accuracy'])
    print(f"\n🏆 Melhor desempenho: {best_result['provider_name']} com {best_result['accuracy']:.2%} de acurácia")
else:
    print("❌ Nenhum resultado válido para comparar.")

📋 COMPARAÇÃO DE RESULTADOS
               Provider        Tipo                                         Modelo  Total  Acertos Acurácia Tempo (s)
        MedGemma-27B-Q8      ollama              jwang580/medgemma_27b_q8_0:latest     20       12   60.00%     306.9
Claude-Sonnet-4-Bedrock aws_bedrock global.anthropic.claude-sonnet-4-20250514-v1:0     20        0    0.00%       2.6
  Maritaca-Sabiazinho-3    maritaca                                   sabiazinho-3     20        0    0.00%       1.5

🏆 Melhor desempenho: MedGemma-27B-Q8 com 60.00% de acurácia


## Geração de Relatórios Detalhados

Geramos relatórios CSV e HTML para cada provider testado.

In [20]:
# Generate detailed reports for each provider
for result in all_results:
    provider_name = result['provider_name'].replace(' ', '_').replace('-', '_')
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # CSV report
    csv_filename = f"test_results_{provider_name}_{timestamp}.csv"
    report_gen = ReportGenerator(output_path=csv_filename)
    report_gen.add_results(result['results'])
    report_gen.save_csv()
    
    # HTML report
    html_filename = f"test_results_{provider_name}_{timestamp}_report.html"
    report_gen.save_html_report(path=html_filename, model_name=result['model'])
    
    print(f"📄 Relatórios gerados para {result['provider_name']}:")
    print(f"   CSV: {csv_filename}")
    print(f"   HTML: {html_filename}")

print("\n✅ Todos os relatórios foram gerados com sucesso!")

CSV salvo em: /Users/filipelopes/Desktop/Development/notebooks/noharm-maritaca/healthbench-br/test_results_MedGemma_27B_Q8_20251004_073005.csv
Relatório HTML salvo em: /Users/filipelopes/Desktop/Development/notebooks/noharm-maritaca/healthbench-br/test_results_MedGemma_27B_Q8_20251004_073005_report.html
📄 Relatórios gerados para MedGemma-27B-Q8:
   CSV: test_results_MedGemma_27B_Q8_20251004_073005.csv
   HTML: test_results_MedGemma_27B_Q8_20251004_073005_report.html
CSV salvo em: /Users/filipelopes/Desktop/Development/notebooks/noharm-maritaca/healthbench-br/test_results_Claude_Sonnet_4_Bedrock_20251004_073005.csv
Relatório HTML salvo em: /Users/filipelopes/Desktop/Development/notebooks/noharm-maritaca/healthbench-br/test_results_Claude_Sonnet_4_Bedrock_20251004_073005_report.html
📄 Relatórios gerados para Claude-Sonnet-4-Bedrock:
   CSV: test_results_Claude_Sonnet_4_Bedrock_20251004_073005.csv
   HTML: test_results_Claude_Sonnet_4_Bedrock_20251004_073005_report.html
CSV salvo em: /Use

## Análise Detalhada por Categoria

Analisamos o desempenho por arquivo e título das questões.

In [ ]:
# Detailed analysis by category
if all_results:
    print("📊 ANÁLISE POR CATEGORIA")
    print("=" * 50)
    
    for result in all_results:
        print(f"\n{result['provider_name']} ({result['model']})")
        print("-" * 40)
        
        # Group by arquivo
        by_file = {}
        for eval_result in result['results']:
            if eval_result.arquivo not in by_file:
                by_file[eval_result.arquivo] = {'total': 0, 'correct': 0}
            by_file[eval_result.arquivo]['total'] += 1
            if eval_result.correta:
                by_file[eval_result.arquivo]['correct'] += 1
        
        print("Por arquivo:")
        for arquivo, stats in by_file.items():
            accuracy = stats['correct'] / stats['total'] if stats['total'] > 0 else 0
            print(f"  {arquivo}: {accuracy:.2%} ({stats['correct']}/{stats['total']})")
        
        # Group by titulo
        by_title = {}
        for eval_result in result['results']:
            if eval_result.titulo not in by_title:
                by_title[eval_result.titulo] = {'total': 0, 'correct': 0}
            by_title[eval_result.titulo]['total'] += 1
            if eval_result.correta:
                by_title[eval_result.titulo]['correct'] += 1
        
        print("\nPor categoria:")
        for titulo, stats in sorted(by_title.items()):
            accuracy = stats['correct'] / stats['total'] if stats['total'] > 0 else 0
            print(f"  {titulo}: {accuracy:.2%} ({stats['correct']}/{stats['total']})")

print("\n✅ Análise completa finalizada!")

## Resumo Final

Sumário dos testes realizados com as primeiras 20 perguntas.

In [ ]:
print("🎯 RESUMO FINAL DO TESTE")
print("=" * 50)
print(f"Dataset testado: Primeiras 20 perguntas do HealthBench-BR")
print(f"Providers testados: {len(all_results)} de {len(active_providers)} ativos")
print(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if all_results:
    print("\nResultados:")
    for result in sorted(all_results, key=lambda x: x['accuracy'], reverse=True):
        print(f"  {result['provider_name']}: {result['accuracy']:.2%} ({result['correct']}/20)")
    
    avg_accuracy = sum(r['accuracy'] for r in all_results) / len(all_results)
    print(f"\nAcurácia média: {avg_accuracy:.2%}")

print("\n✨ Teste concluído com sucesso!")
print("\nPara executar o teste completo com todo o dataset, use:")
print("python evaluate.py --provider <provider> --model <model> --html_report")